## 16. Static analysis vs. tests

**Static analysis** tools examine code *without running it*:
* **mypy / pyright** – type errors, unreachable branches.
* **ruff / flake8** – style and simple bug patterns.
* **bandit** – security smells (insecure hash, subprocess shell=True).

They complement tests by catching an entire class of issues instantly (e.g., misspelled attribute) before runtime. CI should run linters *before* test suite to fail fast.

```bash
mypy src/          # type checking
ruff check src/    # lint & style
bandit -r src/     # security scan
```

### Quick check

1. True / False mypy requires executing the code.

2. Which tool flags insecure functions by AST analysis?
  a. bandit  b. pytest

<details><summary>Answer key</summary>

1. **False** – purely static.
2. **a**.

</details>

## 17. Logging for test diagnosability

Rich logs make failures easier to debug. In pytest, use **`caplog` fixture** to capture output and assert on messages. Set log level so noisy DEBUG doesn’t flood CI; turn up when reproducing locally.

```python
import logging, pytest
logger = logging.getLogger('demo')

def work(x):
    if x < 0:
        logger.error('negative!')
        raise ValueError
    return x*2

def test_logs(caplog):
    with caplog.at_level(logging.ERROR):
        with pytest.raises(ValueError):
            work(-1)
        assert 'negative!' in caplog.text
```

### Quick check

1. `caplog.at_level(logging.INFO)` affects:
  a. global root logger  b. only captured context

2. True / False Logging calls slow tests significantly and should be disabled.

<details><summary>Answer key</summary>

1. **b**.
2. **False** – negligible cost, provides context.

</details>

## 18. Debugging with breakpoints

* **`pdb.set_trace()`** – drop into interactive console.
* **`pytest --pdb`** – auto‑enter debugger on first failure.
* VS Code / PyCharm – graphical breakpoints, variable watch.

Tip: use `breakpoint()` (built‑in) which honours `PYTHONBREAKPOINT` env var.

```python
def buggy():
    x = 1
    breakpoint()  # inspect variables here
    return 1/0
```

### Quick check

1. `breakpoint()` calls which debugger by default?
  a. pdb  b. ipdb

2. True / False `pytest -x --pdb` stops after first failure and opens debugger.

<details><summary>Answer key</summary>

1. **a**.
2. **True**.

</details>

## 19. Tracing execution

Levels:
* **print‑debugging** – quick but pollutes code.
* **`logging.debug`** – toggle via level; better.
* **`trace` / `sys.settrace`** – line‑by‑line profiler/tracer (coverage uses this).
External tracers like **`py-spy`** or **`viztracer`** record call stacks without code mods.

```bash
py-spy record -o profile.svg -- python myscript.py
```

### Quick check

1. `logging.debug` messages show when root level is set to:
  a. WARNING  b. DEBUG

2. True / False `py-spy` requires restarting Python with special flags.

<details><summary>Answer key</summary>

1. **b**.
2. **False** – attaches to running process.

</details>

## 20. Binary search for a failing commit (`git bisect`)

`git bisect` automates finding the exact commit that introduced a bug:
1. `git bisect start`.
2. `git bisect bad` (current) / `git bisect good <hash>`.
3. Git checks out midpoint; you run tests; mark good/bad.
4. Repeat until single culprit commit.

Automate with `git bisect run pytest` for non‑flaky failures.

```bash
git bisect start
git bisect bad          # failing HEAD
git bisect good v1.0.0  # known good tag
git bisect run pytest -q
```

### Quick check

1. Bisect reduces search space:
  a. linearly  b. logarithmically

2. True / False `git bisect reset` returns to original branch.

<details><summary>Answer key</summary>

1. **b**.
2. **True**.

</details>